# Konvertering av Bergens registeret



In [1]:
import polars as pl

In [2]:
'''
Medlemsnummer intervall fra 11000 til 16000.
Medlemsnummer i bergens registeret strekker seg mellom 1 og 6000
I Cornerstone er det ledige nummer intervall mellom 11000 og 16000
For å få de gamle medlemsnummer innenfor intervallet så plusser vi 11000 til gammelt medlemsnummer
Bergen hadde 282 medlemme ved årskiftet til 2025
'''
RANGE_START: int = 11000

In [3]:
# Denne filen er basert på "qryInnmeldte" i Access database og menyvalg "Medlemsregister" er sansynligvis basert på denne
df = pl.read_excel("tblInnmeldte 2025-06-12.xlsx")
df

Could not determine dtype for column 41, falling back to string
Could not determine dtype for column 42, falling back to string


MedlemID,Innmeldingsdato,Er den innmeldte døpt ?,Menighet,Fødested,Dåpsdato,Ant barn under 15 år,Dåpskirke/sted,Fødselsdato,Personnummer,EFTERNAVN,FORNAVN,Stilling,ADRESSE,E-postadresse,POSTNR,Og,SAM_FOR,SAM_EFTER,Utsendelse,Utsendelses kode,Telefon priv,Telefon mob,Sivilstand,Trossamfunn,Gift?,Utsendelsesformat,Ektemakens trossamfunn,Anmerkning,Kategori,Giro,RegDato,SamID,FarID,MorID,Kjønn,Svart?,Faddere,Er mor og far gift?,Prest,Dåpsattest,Søk,UtmeldtDato,DødsDato,Kommune,KID-nr
i64,date,bool,str,str,date,i64,str,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,bool,datetime[ms],i64,i64,i64,str,bool,str,bool,str,bool,str,str,date,str,str
4653,null,false,null,null,null,null,null,null,null,"""Hauger""","""E.""",null,"""_x000D_ Nordahl Rolfsensvei 3…",null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,null,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4654,null,false,null,null,null,null,null,null,null,"""Haugger""","""Elisabeth""",null,null,null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,null,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4655,null,false,null,null,null,null,null,null,null,"""Hauger""","""Elisabeth""",null,"""_x000D_ Nordahl Rolfsensvei 3…",null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2024-12-04 17:24:32,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4656,null,false,null,null,null,null,null,null,null,"""Hauger""","""Elisabeth""",null,"""_x000D_ Nordahl Rolfsensvei 3…",null,"""5094""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2024-12-04 17:25:45,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4628,null,false,"""Bergen""","""Bergen""",null,2,null,1992-10-19,"""19109245731""","""Tangen""","""Andreas""",null,"""Wergelandsalléen 18""",null,"""5067""",null,null,null,null,null,null,null,"""Gift mann""","""Kristensamfunnet""",false,"""E""","""Kristensamfunnet""",null,"""M""",false,2023-06-12 10:14:29,2248,null,null,"""M""",false,null,false,null,false,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4663,2024-12-11,false,null,null,null,null,null,1945-09-02,"""02094546322""","""Bjermeland""","""Trygve""",null,"""Munkebotn 33""",null,"""5036""",null,null,null,null,null,null,null,null,null,false,null,null,null,"""M""",false,2025-02-16 20:09:32,null,null,null,"""M""",false,null,false,null,false,null,null,null,null,null
4664,null,false,null,null,null,null,null,null,null,"""Ruscheinski""","""Gerd Peter""",null,"""Kråkenesdalen 191""",null,"""5153""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2025-05-18 00:00:00,null,null,null,null,false,null,false,null,false,null,null,null,null,null
4665,null,false,null,null,null,null,null,null,null,"""Ruscheinski""","""Gerd Peter""",null,"""Kråkenesdalen 191""",null,"""5153""",null,null,null,null,null,null,null,null,null,false,null,null,null,null,false,2025-05-31 19:40:13,null,null,null,null,false,null,false,null,false,null,null,null,null,null


In [4]:
df.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"
         ).sort("SamID"
               ).filter(pl.col("Personnummer").str.len_chars() == 6)

MedlemID,SamID,FORNAVN,EFTERNAVN,Personnummer
i64,i64,str,str,str
132,null,"""Alfhild Rachlev""","""Dysthe""","""041212"""
216,null,"""Eva Nissen""","""Fett""","""300410"""
336,null,"""Johannes""","""Hertzberg""","""160215"""
418,null,"""Georg Fredrik von""","""Krogh""","""041214"""
454,null,"""Inger Marie von der""","""Lippe""","""120341"""
…,…,…,…,…
3477,3476,"""Stein""","""Befring""","""090754"""
3489,3490,"""Per Walfrid""","""Odén""","""210346"""
690,3513,"""Ingvar""","""Tysvær""","""120659"""


In [5]:

df = df.with_columns(
    # Lager nye medlemsnummere som er tilpasset Cornerstone
    (pl.col("MedlemID") + RANGE_START).alias("MedlemID"),
    (pl.col("SamID") + RANGE_START).alias("SamID"),
    (pl.col("MorID") + RANGE_START).alias("MorID"),
    (pl.col("FarID") + RANGE_START).alias("FarID"),
    # Tar bort Personnummer der det bare er skervet in fødselsdato delen
    # Har sjekket at "Fødselsdato" er satt på alle det gjelder
    pl.when(pl.col("Personnummer").str.len_chars() == 6)
      .then(None)
      .otherwise(pl.col("Personnummer"))
      .alias("Personnummer")
)


In [6]:
df.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"
         ).sort("SamID"
               )#.filter(pl.col("Personnummer").str.len_chars() == 6)

MedlemID,SamID,FORNAVN,EFTERNAVN,Personnummer
i64,i64,str,str,str
15653,null,"""E.""","""Hauger""",null
15654,null,"""Elisabeth""","""Haugger""",null
15655,null,"""Elisabeth""","""Hauger""",null
15656,null,"""Elisabeth""","""Hauger""",null
15630,null,null,null,null
…,…,…,…,…
15547,15604,"""Tone Bell""","""Rysst""","""22055032878"""
15551,15606,"""Tora Døssland""","""Eikeland""","""16065247058"""
13248,15628,"""Katarina""","""Lunde""","""03089142074"""


## Rekkefølge på konverteringen
- Alle som har fødselsnummer, medlemmer med flere
- Alle som er registrert med "MedlemID", "SamID", "MorID", "FarID"
- Alle som har "MedlemID" i "SamID", "MorID", "FarID"
- 
- Det vil være overlappende "MedlemID" for de som er listet over. Få ut en liste og krymp den til disticte verdier av "MedlemID"
- Resterende personer vil ha datakvalitet av varierende kvalitet. Disse konverteres til slutt. Det er resterende som ikke er med i den oppbyggde "MedlemID" listen. Ingen av disse er medlemmer. De har heller ikke forbindelser til andre personer.

In [7]:
df_medlemId = (
    df
    .filter(
        (pl.col("Kategori").str.starts_with("M")) &
        # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer"
        (pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
    )
    .select("MedlemID")
)
df_medlemId

MedlemID
i64
15628
15626
15627
15513
15429
…
15638
15649
15650


In [9]:
df_connectedMedlemID = (
    pl.concat([
        df.filter(pl.col(c).is_not_null()).select(pl.col(c).alias("MedlemID")).unique()
        for c in ["SamID", "MorID", "FarID"]
    ])
    .unique()
)
df_connectedMedlemID

MedlemID
i64
14296
14449
15535
12743
13812
…
12729
13325
15585


In [11]:
# Finner alle ID'er som ikke eksisterer
# 1. Anti-join: IDs in connected that aren't in df
missing_ids = df_connectedMedlemID.join(
    df, 
    on="MedlemID", 
    how="anti"
)
print(missing_ids)
# Sletter alle ID'er som ikke finnes fra vår Connected liste
df_connectedMedlemID = df_connectedMedlemID.join(
    missing_ids,
    on="MedlemID", 
    how="anti"
)
print(df_connectedMedlemID)

shape: (32, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 13618    │
│ 13237    │
│ 11025    │
│ 14526    │
│ 13346    │
│ …        │
│ 12121    │
│ 14363    │
│ 11113    │
│ 15436    │
│ 13325    │
└──────────┘
shape: (202, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 14296    │
│ 14449    │
│ 15535    │
│ 12743    │
│ 13812    │
│ …        │
│ 11212    │
│ 12729    │
│ 15585    │
│ 15529    │
│ 15628    │
└──────────┘


In [12]:
# Oppdaterer df og setter alle ikke eksisterende kontakter til null
df = df.with_columns([
    pl.when(pl.col(c).is_in(missing_ids["MedlemID"].to_list()))
      .then(None)
      .otherwise(pl.col(c))
      .alias(c)
    for c in ["SamID", "MorID", "FarID"]
])


In [ ]:
df


In [19]:
# Alle medlemmer som har kontakter på seg
df_harconnectedMedlemID = (
    df
    .filter(
        pl.col("SamID").is_not_null() |
        pl.col("MorID").is_not_null() |
        pl.col("FarID").is_not_null()
    )
    .select(
        pl.col("MedlemID")
    )
)
df_harconnectedMedlemID

MedlemID
i64
15628
15621
15626
15627
15513
…
15453
15460
15636


In [26]:
# Lager en liste av personer som har god kvalitet på data
# One‑liner using a semi‑join (no need for unique() – keeps only df rows whose MedlemID is in the concatenated list)
df_kvalitetsData = df.join(
    pl.concat([df_harconnectedMedlemID, df_connectedMedlemID, df_medlemId], how="vertical"),
    on="MedlemID",
    how="semi"
)
print(df_kvalitetsData)

# Lager en liste av personer som sjanse for dårligere kvalitet på data
# Option 1: Anti‑join (most idiomatic & performant)
df_looserData = df.join(
    df_kvalitetsData,
    on="MedlemID",
    how="anti"
)
print(df_looserData)

shape: (490, 46)
┌──────────┬────────────┬────────────┬──────────┬───┬────────────┬──────────┬─────────┬────────────┐
│ MedlemID ┆ Innmelding ┆ Er den     ┆ Menighet ┆ … ┆ UtmeldtDat ┆ DødsDato ┆ Kommune ┆ KID-nr     │
│ ---      ┆ sdato      ┆ innmeldte  ┆ ---      ┆   ┆ o          ┆ ---      ┆ ---     ┆ ---        │
│ i64      ┆ ---        ┆ døpt ?     ┆ str      ┆   ┆ ---        ┆ date     ┆ str     ┆ str        │
│          ┆ date       ┆ ---        ┆          ┆   ┆ str        ┆          ┆         ┆            │
│          ┆            ┆ bool       ┆          ┆   ┆            ┆          ┆         ┆            │
╞══════════╪════════════╪════════════╪══════════╪═══╪════════════╪══════════╪═════════╪════════════╡
│ 15628    ┆ null       ┆ false      ┆ Bergen   ┆ … ┆ null       ┆ null     ┆ null    ┆ null       │
│ 15621    ┆ null       ┆ false      ┆ null     ┆ … ┆ null       ┆ null     ┆ null    ┆ null       │
│ 15626    ┆ 2021-12-17 ┆ true       ┆ Bergen   ┆ … ┆ null       ┆ null   